In [1]:
import os
from openpyxl import load_workbook

In [2]:
# project coding spreadsheet
proj_coding_spreadsheet_dir = r'C:\Users\toleary\0_Tasks\MHN\RCP_Evaluation\MHN\import\RSP_Coding\RCP22206_CottageGroveBPC.xlsx' #replace with csv filepath
worksheet_with_ttf2_links = 'ttf2_postprocessing' #replace with worksheet name

# output transaction file location (folder that itin files are located)
batchin_file_dir = r'C:\Users\toleary\0_Tasks\MHN\RCP_Evaluation\MHN\output\final\RCP22206_CottageGroveBPC\transit\700'

In [3]:
# get relevant ttf2 links
links=[]
wb = load_workbook(filename = proj_coding_spreadsheet_dir, read_only=True)
links_ws = wb[worksheet_with_ttf2_links]
for row in links_ws.values:
    if all(type(l)==int for l in row): #remove headers or other non-node rows
        links.append(list(row))
wb.close()

# add reverse direction
links_ba = [[b, a] for pair in links for a, b in [pair]]
links += links_ba

In [4]:
# for file in os.listdir(batchin_file_dir):
#     if 'bus.itinerary' in file:
#         print(f'processing {file}...')
#         with open(os.path.join(batchin_file_dir, file), 'r') as f:
#             for row in f:
#                 if row.startswith('a'): #start of tr line
#                     busline = row
#                     prev_node = None 
#                 elif row.strip().split(' ')[0].isnumeric():
#                     curr_node = row.strip().split(' ')[0]
#                     if prev_node:
#                         curr_link = [int(prev_node), int(curr_node)]
#                         if curr_link in links:
#                             print(f'{busline.strip()} {curr_link}')
#                     else:
#                         prev_node = curr_node

itin_files = [file for file in os.listdir(batchin_file_dir) if 'bus.itinerary' in file]

In [5]:
# Add this cell after your existing cells
def modify_ttf_in_itinerary(file_path, target_links):
    """
    Reads bus itinerary file and changes ttf=1 to ttf=2 for node-to-node links
    that are in the target_links list.
    """
    
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    modified_lines = []
    prev_node = None
    prev_line = None
    for line in lines:
        line_s = line.strip()
        
        # Check if this line contains a node (starts with a number)
        if line_s and line_s.split()[0].isdigit():
            curr_node = int(line_s.split()[0])
            # If we have a previous node, check if this link needs modification
            if prev_node is not None:
                curr_link = [prev_node, curr_node]
                # Check if this link is in our target list
                if curr_link in target_links:
                    # Replace ttf=1 with ttf=2
                    prev_line = prev_line.replace('ttf=1', 'ttf=2')
                    modified_lines[-1] = prev_line
                    # print(f'\tModified link {prev_node} -> {curr_node}: changed ttf=1 to ttf=2')
            
            prev_node = curr_node
        
        elif line_s.startswith('a'):
            # print(line_s)
            prev_node = None
        
        else:
            prev_node = None
            
        prev_line = line
        
        modified_lines.append(line)
    
    # Write back to file
    with open(file_path, 'w') as file:
        file.writelines(modified_lines)
    
    print(f"File modified successfully!")

# Apply the modification
for file in itin_files:
    print(f'Processing {file}...')
    modify_ttf_in_itinerary(os.path.join(batchin_file_dir, file), links)
    
print('All done!')

Processing bus.itinerary_1...
File modified successfully!
Processing bus.itinerary_2...
File modified successfully!
Processing bus.itinerary_3...
File modified successfully!
Processing bus.itinerary_4...
File modified successfully!
All done!


In [ ]:
links